In [2]:
from sklearn.neural_network import MLPClassifier as mlpc
from sklearn.preprocessing import StandardScaler,QuantileTransformer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, roc_auc_score

import mltools as ml

In [3]:
X = np.genfromtxt('data/X_train.txt', delimiter=',')
Y = np.genfromtxt('data/Y_train.txt', delimiter=',')
np.random.seed(0)

X,Y = ml.shuffleData(X,Y)
X = X[:,:41]

In [5]:
scaler = StandardScaler()
transformer = QuantileTransformer()

Xtr, Xva, Ytr, Yva = ml.splitData(X,Y,0.7)
Xtr_scaled = scaler.fit_transform(Xtr, Ytr)
Xva_scaled = scaler.fit_transform(Xva,Yva)
Xtr_transformed = transformer.fit_transform(Xtr)
Xva_transformed = transformer.fit_transform(Xva)

In [58]:
learner = mlpc(max_iter=1000, alpha=0.001)#no weights first.
learner.fit(Xtr_transformed,Ytr) #the training

MLPClassifier(alpha=0.001, max_iter=1000)

In [59]:
Yva_hat = learner.predict(Xva_transformed)

print(f" auc: {roc_auc_score(Yva, Yva_hat)} | mse: {mean_squared_error(Yva, Yva_hat)}")

 auc: 0.6083760807846174 | mse: 0.3915581499775483


`alpha`= 0.001 is the best alpha we've discovered.
`solver` should NOT be `"lbfgs".
`random_state` should be kept to `None` (default).

In [18]:
learner = mlpc(max_iter=10000, alpha=0.001, hidden_layer_sizes=(100,100,100), activation='tanh')#no weights first.
learner.fit(Xtr_transformed,Ytr) #the training

MLPClassifier(activation='tanh', alpha=0.001,
              hidden_layer_sizes=(100, 100, 100), max_iter=10000)

In [19]:
Yva_hat = learner.predict(Xva_transformed)

print(f" auc: {roc_auc_score(Yva, Yva_hat)} | mse: {mean_squared_error(Yva, Yva_hat)}")

 auc: 0.5767058652729384 | mse: 0.42343960484957344


In [23]:
from sklearn.model_selection import GridSearchCV

parameter_space = {
    'hidden_layer_sizes': [(100,),(10,10,10), (100,100,100), (30,50,30)],
    'activation': ['tanh','relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.001,0.05],
    'learning_rate': ['constant','adaptive']
}

learner = mlpc(max_iter=10000)

clf = GridSearchCV(learner,parameter_space,n_jobs=-1,cv=5)

clf.fit(Xtr_transformed, Ytr)

GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=10000), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'], 'alpha': [0.001, 0.05],
                         'hidden_layer_sizes': [(100,), (10, 10, 10),
                                                (100, 100, 100), (30, 50, 30)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']})

In [24]:
print('Best parameters: ', clf.best_params_)

Best parameters:  {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (10, 10, 10), 'learning_rate': 'adaptive', 'solver': 'adam'}
